In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import re
#In order to ignore false-positive chained assignment warning
pd.options.mode.chained_assignment = None

In [2]:
book_dtf = pd.read_csv("C:/Users/joann/Downloads/newProject/Data_Cleaning/Cleaned_Data.csv")

#Resseting the index
book_dtf = book_dtf.reset_index(drop = True)

In [3]:
#Filtering reviews that is written in English only

#And to do so, we'll be using the fasttext library
!pip install fasttext
import fasttext

In [4]:
#Deleting the previous indexing
del book_dtf["Unnamed: 0"]
book_dtf.head()

,User_ID,Book_ID,Stars_Ratings,Review,Date_Added,Likes_Count,Num_Review,Num_Ratings,Average_Ratings,Author_Name,Combined_Likes_Count
0,e88c751829dc59cef8e3f23bf10e0f13,1,5,"""I am the Half-blood prince"". \n Read to find ...",Sat Nov 26 10:52:38 -0800 2016,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
1,bd12db2c6bb0e2e26e2b1868a1463b68,1,5,The insights into the life of Tom Riddle do no...,Fri Aug 30 18:56:16 -0700 2013,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
2,5c0a76a445829482c8bbc8243734795a,1,3,"In no way do I diss Harry Potter, but I must s...",Sun Apr 19 16:46:01 -0700 2009,2,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
3,c96c64adc01b167628026634543278e9,1,5,This was my favorite HP book so far! What a ch...,Tue Jan 06 15:26:36 -0800 2015,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
4,4af2b6a2ed4e32ee9f12f7240c3b6df8,1,5,One of the most moving of the series. It is in...,Thu Jun 23 08:09:30 -0700 2016,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439


In [5]:
book_dtf.shape

(2309970, 11)

In [6]:
#Loading the pre-trained model
model = fasttext.load_model('lid.176.bin')

In [7]:
#Remove the \n character from the reviews and replace it with spacebar, because fasttext can't predict with \n character
#Reviews thus will only be one paragraph instead of multiple paragraphs

list_of_reviews = book_dtf["Review"].str.replace('\n','').to_list()
prediction = model.predict(list_of_reviews)

In [8]:
#Prediction return two values:
    #If review is in English, then "__label__en" will be returned, otherwise, "__label__xx" where the xx will be replaced by the predicted langugage
    #It will also return the probability of how sure they are that the text is the one that has been predicted

#Thus, retain every review with "__label__en" with probability of it being English being more than 80%. The index of these reviews are kept in the list english_reviews_index
english_reviews_index = []
index = 0
for i in range(len(prediction[0])):
    if prediction[0][i][0] == "__label__en" and prediction[1][i][0] > 0.80:
        english_reviews_index.append(i)
        
print(len(english_reviews_index))

2029087


In [9]:
#We keep the data whose index is in the english_reviews_index
#Index is reseted, so that the old index won't be used in the new dataframe. Drop is set to True so that the old index isn't saved as a column

book_dtf = book_dtf.iloc[english_reviews_index,].reset_index(drop=True)
print(book_dtf.shape)

(2029087, 11)


In [10]:
book_dtf.to_csv('Cleaned_Data.csv', encoding='utf-8')